# Wearable Stress and Affect Detection (WESAD) Feature Extraction & Clustering

[UCI Link](https://archive.ics.uci.edu/ml/datasets/WESAD+%28Wearable+Stress+and+Affect+Detection%29#)

''' Philip Schmidt, Attila Reiss, Robert Duerichen, Claus Marberger and Kristof Van Laerhoven. 2018. Introducing WESAD, a multimodal dataset for Wearable Stress and Affect Detection. In 2018 International Conference on Multimodal Interaction (ICMI '18), October 16-20, 2018, Boulder, CO, USA. ACM, New York, NY, USA, 9 pages. '''

This dataset is part of the UCI ML Data repository and contains high granularity data (700 Hz) of 15 test subjects from chest worn sensors (RespiBAN) in the form of:

- Electrocardiography (ECG)
- Electrodermal Activity (EDA)
- Electromyography (EMG)
- Body Temp (Temp)
- Accelorometer (ACC)
- Respiration % (Resp)

Contains data at lower granularity from wrist worn (non dominant) Empatica device in the form of:

- Accelorometer (ACC)
- Blood Volume Pulse (BVP)
  - This feature wasn't actually explained in the connected README for the dataset, so using the information [here](https://biosignalsplux.com/products/sensors/blood-volume-pulse.html) which is the same source as for the other data
- Electrodermal Activity (EDA)
- Body Temp (Temp)

Wearable data generation has exploded in recent years, and with it the analysis of it. Time series data can yield very interesting insights and can paint a picture of people's health that they would not be able to see themselves.

Exploratory analysis was done in separate notebooks; this notebook will specifically look at feature extraction from the original dense data then clustering based on those features. Specifically, this notebook will leverage the R code found in this 

# The features (Based on original paper [here](https://www.researchgate.net/publication/220451959_Characteristic-Based_Clustering_for_Time_Series_Data))

The following features will be extracted from each time series as inputs for clustering:

**On Original Data:**
- Frequency
- Trend
- Seasonality
- Autocorrelation
- Non-linearity
- Skewness
- Kurtosis
- Hurst
- Lyapunov

**On Decomposed Data (with trend & seasonality removed):**
- Decomposed Autocorrelation
- Decomposed Non-linearity
- Decomposed Skewness
- Decomposed kurtosis

More on each of these features is contained in the explanation markdown file.

# [Original R Code](https://www.r-bloggers.com/measuring-time-series-characteristics/)

The above link is to the R code that this code will be based off of.

After seeing the above link, I tried my hand at turning this code into Python. I was able to get slightly more than half of the above features equivalent to the results from the R code. Some of the features simply wouldn't match up, and others simply could not be computed in Python due to lack of supporting packages/documentation.

## In R as well (since I can't get it to work in Python)

In [1]:
# Requires some manual work before these code blocks will work
# 1. Set R_HOME variable (can be found by typing ".Library" into R Studio)
# 2. The output from the above is the "library" subdirectory of the R_HOME directory established by R Studio
# 3. Take the output minus "/library" at the end, and save this to R_HOME in a terminal
# 4. Run pip3 install rpy2 (connects to R_HOME at install)
# Will make an init.sh file to run all the necessary steps for this + other manual setup steps

# Sources: https://stackoverflow.com/questions/17573988/r-home-error-with-rpy2
# https://stackoverflow.com/questions/47585718/rpy2-installed-but-wont-run-packages
# https://stackoverflow.com/questions/24880493/how-to-find-out-r-library-location-in-mac-osx/24880594
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
from rpy2.robjects import r, pandas2ri, numpy2ri, Formula
from rpy2.robjects.vectors import IntVector,FloatVector
# Load necessary R packages
rtseries = importr('tseries')
rbase = importr('base')
rstats = importr('stats')
rfracdiff=importr('fracdiff')
rutils=importr('utils')
rmgcv=importr('mgcv')

# "Activating" pandas2ri and numpy2ri with .activate() method

Typically, the above libraries are "activated" before use to make working with the python dataframes easier. For some reason, running the above commands blocks me from being able to create R time series objects. Trying to create time series objects just leads to numpy arrays being created(used the following [resource](https://www.r-bloggers.com/using-r-in-python-for-statistical-learning-data-science-2/) to help).

Example output when pandas2ri and numpy2ri are NOT activated:
```
rbase.set_seed(123) # reproducibility seed
x = r.ts(r.rnorm(n=10)) # simulate the time series
print(x)

Time Series:
Start = 1 
End = 10 
Frequency = 1 
 [1] -0.56047565 -0.23017749  1.55870831  0.07050839  0.12928774  1.71506499
 [7]  0.46091621 -1.26506123 -0.68685285 -0.44566197
```

This is the format we want; otherwise certain R command will not work because they explicitly want a time series object (seen above) as input.

Example output when pandas2ri and numpy2ri ARE activated:

```
pandas2ri.activate()
numpy2ri.activate()
rbase.set_seed(123) # reproducibility seed
x = r.ts(r.rnorm(n=10)) # simulate the time series
print(x)

[-0.56047565 -0.23017749  1.55870831  0.07050839  0.12928774  1.71506499
  0.46091621 -1.26506123 -0.68685285 -0.44566197]
```

I have not actually run the code myself because once the "activate" methods are called, I have not found a way to be able to create time series objects without restarting the whole notebook/shell. You are welcome to try out the commands for yourself.

## Get frequency

The R code has three functions, one of which is to automatically determine the frequency of the data by using AR spectrum modeling on the data as seen below:


```
# Function to find the frequency of the time series data inputted
find.freq <- function(x)
{
  n <- length(x)
  spec <- spec.ar(c(na.contiguous(x)),plot=FALSE)
  if(max(spec$spec)>10) # Arbitrary threshold chosen by trial and error.
  {
    period <- round(1/spec$freq[which.max(spec$spec)])
    if(period==Inf) # Find next local maximum
    {
      j <- which(diff(spec$spec)>0)
      if(length(j)>0)
      {
        nextmax <- j[1] + which.max(spec$spec[j[1]:500])
        if(nextmax <= length(spec$freq))
          period <- round(1/spec$freq[nextmax])
        else
          period <- 1
      }
      else
        period <- 1
    }
  }
  else
    period <- 1

  return(period)
}
```

### Python Version (doesn't match up)

In [2]:
## First find frequency
import numpy as np
import pandas as pd
from scipy.signal import periodogram
def find_freq(x):
    # Use an iterative function to automagically determine the frequency of the time series data
    # Takes in a single column of a pandas DataFrame as a univariate time series
    
    n = len(x)
    # Now estimate the spectral density of the time series via AR fit
    # Two ways: numpy fft method or scipy signal method
    # Method #1: numpy fft method (https://stackoverflow.com/questions/15382076/plotting-power-spectrum-in-python)
    '''
    pow_np = np.abs(np.fft.fft(x))**2
    time_step = 1 / n
    freqs = np.fft.fftfreq(n, time_step)
    idx = np.argsort(freqs)
    plt.plot(freqs[idx], ps[idx])
    '''
    
    # Method #2: scipy signal method via density or spectrum (takes an optional second frequency parameter)
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.periodogram.html
    #f, pow_scipy = signal.periodogram(x,scaling='density')
    f, pow_scipy = periodogram(x,scaling='spectrum')
    
    # Iterate through frequencies 
    freq = f
    power = pow_scipy
    if max(power) > 10: # Arbitrary threshold chosen by trial and error.
        # The power might be a huge number way of out the index bounds, so pick max index if so
        power_idx = min(int(round(max(power))),(len(power)-1))
        freq_idx = min(int(round(power[power_idx])),len(freq)-1)
        period = 1/freq[freq_idx]
        # If period is infinity, find next local maximum
        if period == np.Inf:
            j = pd.Series(power).diff() > 0
            if len(j) > 0:
                nextmax = j[1] + power[int(round(max(power[1:])))]
                if (nextmax <= len(freq)):
                    period = int(round(1/freq[int(round(nextmax))]))
                else:
                    period = 1
            else:
                period = 1
        else:
            period = int(round(period))
    else:
        period = 1
    
    return period

### R Code in Python

In [3]:
def find_freq_r(x):
    # Same as above function, but using the R code directly via rpy2
    n = len(x)
    #spec = rstats.spec_ar(r.ts(na_contiguous(x)),plot=False)
    spec = rstats.spec_ar(x,plot=False)
    # This returns a "ListVector" with the following items, which can be accessed via list indexing:
    '''
    Example output
    x = r.ts(r.rnorm(n=30)) # simulate the time series
    spec = rstats.spec_ar(x,plot=False)
    print(spec.names)
    
    [1] "freq"   "spec"   "coh"    "phase"  "n.used" "series" "method"
    '''
    spec_vals = spec[1]
    spec_freq = spec[0]

    if max(spec_vals) > 10:
        #period <- round(1/spec$freq[which.max(spec$spec)])
        denom = spec_freq[list(rbase.which_max(spec_vals))[0] - 1]
        if denom != 0:
            period = round(1/denom)
        else: # Means we end up with infinity as a result, so evaluate additional code block
            series = pd.Series(rbase.diff(spec_vals))
            j = series[series > 0].reset_index(drop=True)
            if len(j) > 0:
                #nextmax <- j[1] + which.max(spec$spec[j[1]:500])
                nextmax = j[0] + rbase.which_max(spec_vals[int(j[0]):500])
                if nextmax.item() - 1 <= len(spec_freq):
                    denom = spec_freq[round(nextmax.item() - 1)]
                    if denom != 0:
                        period = round(1/denom)
                    else:
                        period = 1
                else:
                    period = 1
            else:
                period = 1
    else:
        period = 1
    
    return int(period)

## Decompose function

The second of the three functions in the R code takes a time series object and extracts the trend and seasonality 

In [4]:
from scipy.stats import boxcox
#from statsmodels.gam.api import GLMGam, BSplines
from statsmodels.gam.generalized_additive_model import GLMGam
from statsmodels.gam.smooth_basis import BSplines,CyclicCubicSplines
import statsmodels.api as sm
#from statsmodels.gam.generalized_additive_model import GLMGam

def na_contiguous(x):
    # Recreate na.contiguous function in R since this is used frequently
    # This takes a series object with a time index and finds the longest consecutive stretch of non-missing values
    # https://stackoverflow.com/questions/41494444/pandas-find-longest-stretch-without-nan-values
    # And then return the shortened dataframe with all non-null values
    values = x.values 
    mask = np.concatenate(( [True], np.isnan(values), [True] ))  # Mask
    start_stop = np.flatnonzero(mask[1:] != mask[:-1]).reshape(-1,2)   # Start-stop limits
    start,stop = start_stop[(start_stop[:,1] - start_stop[:,0]).argmax()]  # Get max interval, interval limits
    contiguous = x.iloc[start:stop]
    return contiguous

def decompose(x, transform = True):
    # Decompose data into trend, seasonality and randomness
    # Accepts a pandas series object with a datetime index
    if (transform and min(x.dropna()) >= 0):
        # Transforms data and finds the lambda that maximizes the log likelihood 
        # R version has above method and method that minimizes the coefficient of variation ("guerrero")
        x_transformed, var_lambda = boxcox(na_contiguous(x),lmbda = None)
        x_transformed = pd.Series(x_transformed,index=na_contiguous(x).index)
    
    else:
        x_transformed = x
        var_lambda = np.nan
        transform = False
    
    # Seasonal data 
    # In R code, we find the number of samples per unit time below (should be 1 every time)
    # Tried implementing this in Python, but just went with the R code instead
    # This is supposed to be "> 1" but all data results in a frequency of 1
    # All frequency results in R equal 4, meaning this code block gets evaluated every time in R
    # So this code block should always be evaluated as well
    freq = rstats.frequency(r.ts(FloatVector(x_transformed)))
    if int(list(freq)[0]) == 1:
        # Decompose
        stl = sm.tsa.seasonal_decompose(na_contiguous(x_transformed),period=1)
        #stl = rstats.stl(na_contiguous(x_transformed),s_window='periodic')
        # When I try to use above function, I get this:
        '''
        R[write to console]: Error in (function (x, s.window, s.degree = 0, t.window = NULL, t.degree = 1,  : 
  series is not periodic or has less than two periods
        '''
        trend = stl.trend
        seasonality = stl.seasonal
        remainder = x_transformed - trend - seasonality

    else:
        # Nonseasonal data
        trend = pd.Series(np.nan, index=x_transformed.index)
        time_index = pd.Index([i for i in range(1,len(x_transformed)+1)])
        # Python specific
        bs = BSplines(time_index, df=[12, 10], degree=[3, 3])
        cs = CyclicCubicSplines(time_index,df=[3,3])
        alpha = np.array([218.338888])
        gam = GLMGam(x_transformed, smoother=cs, alpha=alpha).fit()
        #trend.loc[~x_transformed.isnull()] = gam.fittedvalues
        
        # R Code
        fmla = Formula('x ~ s(tt)')
        env = fmla.environment
        env['tt'] = time_index
        env['x'] = x_transformed
        trend.loc[~x_transformed.isnull()] = rstats.fitted(rmgcv.gam(fmla))
        seasonality = pd.Series(np.nan, index=x_transformed.index)
        remainder = x_transformed - trend
    
    return_dct = {
        'x': x_transformed,
        'trend': trend,
        'seasonality': seasonality,
        'remainder': remainder,
        'transform': transform,
        'lambda': var_lambda,
    }
    
    return return_dct

### Transformation (standardization) functions

All of the features are standarized according to one of two of the following transformations:

```
# Functions to map all the features onto a [0,1] scale
# f1 maps [0,infinity) to [0,1]
f1 <- function(x,a,b)
{
  eax <- exp(a*x)
  if (eax == Inf)
    f1eax <- 1
  else
    f1eax <- (eax-1)/(eax+b)
  return(f1eax)
}

# f2 maps [0,1] onto [0,1]
f2 <- function(x,a,b)
{
  eax <- exp(a*x)
  ea <- exp(a)
  return((eax-1)/(eax+b)*(ea+b)/(ea-1))
}
```

In [5]:
import math
def f1_transformation(x, a, b):
    eax = math.exp(a * x)
    if eax == np.Inf:
        f1_eax = 1
    else:
        f1_eax = (eax-1)/(eax+b)
    return f1_eax

def f2_transformation(x, a, b):
    eax = math.exp(a*x)
    ea = math.exp(a)
    return((eax-1)/(eax+b)*(ea+b)/(ea-1))

## Measure Calculations

Using the various functions above, we calculate the final measures as seen in the code below:

In [6]:
from scipy.special import inv_boxcox
from statsmodels.stats.diagnostic import acorr_ljungbox
from pypr.stattest.ljungbox import boxpierce
def calculate_measures(x):
    # Save ts version of our data for some of the below functions
    rbase.set_seed(123) # reproducibility seed
    x_ts_contiguous = r.ts(FloatVector(na_contiguous(x)))
    
    N = len(x)
    freq = find_freq_r(x_ts_contiguous)
    print('Frequency calculated')
    fx = (math.exp((freq-1)/50)-1)/(1+math.exp((freq-1)/50))
    
    # Decomposition
    decomp_x = decompose(x)
    print('Decomposition complete')
    
    # Adjust data
    # Unfortunately it looks like frequency is calculated a different way in the decompose function
    # Thus there are users for which this function is evaulated when 'seasonality' is null
    # Going to add an extra check to make sure to not evaluate this if all the values are null
    if freq > 1 and (not decomp_x['seasonality'].isnull().all()):
        fit = decomp_x['trend'] + decomp_x['seasonality']
    else:
        # Nonseasonal data
        fit = decomp_x['trend']
    adj_x = decomp_x['x'] - fit + np.mean(decomp_x['trend'].dropna())
    print('Adj x complete')
    
    # Backtransformation of adjusted data
    if decomp_x['transform']:
        # The below line of code doesn't work for some reason
        #t_adj_x = inv_boxcox(adj_x.values, decomp_x['lambda'])
        # Use actual formula instead (but do inverse because we're solving for x)
        '''
        The Box-Cox transform is given by:

            y = (x**lmbda - 1) / lmbda,  for lmbda > 0
                log(x),                  for lmbda = 0
        '''
        if decomp_x['lambda'] == 0:
            # Assuming base of 10 (x = 10^y)
            t_adj_x = 10 ** adj_x
        else:
            # x = ((y * lambda) + 1) ^ (1/lambda)
            t_adj_x = ((adj_x * decomp_x['lambda']) + 1) ** (1/decomp_x['lambda'])
    else:
        t_adj_x = adj_x
    
    print('Calculating trend/seasonal measures')
    # Trend and seasonal measures
    v_adj = np.var(adj_x.dropna())
    threshold = 0.00000000001
    if(freq > 1):
        detrend = decomp_x['x'] - decomp_x['trend']
        deseason = decomp_x['x'] - decomp_x['seasonality']
        
        if np.var(deseason.dropna()) < threshold:
            trend = 0
        else:
            trend = max(0,min(1,1-(v_adj/np.var(deseason.dropna()))))
        if np.var(detrend.dropna()) < threshold:
            seasonality = 0
        else:
            seasonality = max(0,min(1,1-(v_adj/np.var(detrend.dropna()))))
    else:
        # Nonseasonal data
        if np.var(decomp_x['x'].dropna()) < threshold:
            trend = 0
        else:
            trend = max(0,min(1,1-(v_adj/np.var(decomp_x['x'].dropna()))))
        seasonality = 0
    
    measures = [fx,trend,seasonality]
    
    # Measures on original data
    xbar = np.mean(x.dropna())
    std = np.std(x.dropna())
    print('Calculating serial correlation')
    ### THIS IS WHERE THE ISSUE IS ###
    
    # Serial correlation (method 1)
    #Had to fix stattest module in pypr package via: https://gist.github.com/betterxys/1def38e1fcbb7f3b2dab2393bcea52f0
    max_lag = 10
    bp = boxpierce(x, lags=max_lag)
    Q = bp / (N*max_lag)
    fQ = f2_transformation(Q,7.53,0.103)

    # Serial correlation (make sure box pierce statistic is returned as well)
    # Method 2, but ends up never finishing :(
    #lbvalue, pvalue, bpvalue, bppvalue = acorr_ljungbox(x, lags=max_lag, boxpierce=True)
    # The above returns values for each lag, so just grab the final value
    #Q = bpvalue[-1] / (N*max_lag)
    #fQ = f2_transformation(Q,7.53,0.103)
    
    # Nonlinearity (THIS REQUIRES THE TIMESERIES OBJECT VERSION OF OUR DATA)
    print('Calculating non linear test')
    non_linear_test = rtseries.terasvirta_test_ts(x_ts_contiguous,type = "Chisq")
    '''
    x = r.ts(r.rnorm(n=30)) # simulate the time series
    non_linear_test = rtseries.terasvirta_test_ts(x,type = "Chisq")
    print(non_linear_test.names)
    [1] "statistic" "parameter" "p.value"   "method"    "data.name" "arguments"
    '''
    # Grab the first value as seen above
    p = list(non_linear_test[0])[0]
    fp = f1_transformation(p,0.069,2.304)
    
    print('Calculating skew + kurtosis')
    # Skewness
    skew = abs(np.mean((x.dropna()-xbar) ** 3)/std ** 3)
    fs = f1_transformation(skew,1.510,5.993)
    
    # Kurtosis
    kurtosis = np.mean((x.dropna()-xbar) ** 4)/std ** 4
    fk = f1_transformation(kurtosis,2.273,11567)
    
    # Hurst=d+0.5 where d is fractional difference
    print('Calculting hurst')
    hurst = rfracdiff.fracdiff(x_ts_contiguous,0,0)
    '''
    x = r.ts(r.rnorm(n=30)) # simulate the time series
    hurst = rfracdiff.fracdiff(x_ts_contiguous,0,0)
    print(hurst.names)
     [1] "log.likelihood"  "n"               "msg"             "d"              
     [5] "ar"              "ma"              "covariance.dpq"  "fnormMin"       
     [9] "sigma"           "stderror.dpq"    "correlation.dpq" "h"              
    [13] "d.tol"           "M"               "hessian.dpq"     "length.w"       
    [17] "residuals"       "fitted"          "call"     
    '''
    # Grab the fourth value in the hurst variable
    H = list(hurst[3])[0] + 0.5
    
    # Lyapunov Exponent
    print('Calculating Lyapunov Exponent')
    '''
    if freq > (N-10):
        # There is insufficient data, declare this variable as none
        fLyap = None
    else:
        Ly = np.zeros(N-freq)
        for i in range(0,(N-freq)):
            diffs = abs(x.iloc[i] - x)
            date_idx = diffs.sort_values().index
            int_idx = pd.Index([diffs.index.get_loc(date) for date in date_idx])
            idx = int_idx[int_idx < (N-freq)]
            j = idx[1]
            try:
                Ly[i] = math.log(abs((x.iloc[i+freq] - x.iloc[j+freq])/(x.iloc[i]-x.iloc[j]))) / freq
            except ValueError: # domain error, means log(0) was taken
                Ly[i] = 0
            if(np.isnan(Ly[i]) or (Ly[i] == np.Inf) or (Ly[i] == -np.Inf)):
                Ly[i] = np.nan
        Lyap = np.mean(Ly[~np.isnan(Ly)])
        fLyap = math.exp(Lyap) / (1+math.exp(Lyap))
    '''
    fLyap = None
    
    measures = measures + [fQ,fp,fs,fk,H,fLyap]
    
    # Measures on adjusted data
    print('Calculating adjusted data measures')
    xbar = np.mean(t_adj_x.dropna())
    std = np.std(t_adj_x.dropna())

    # Serial correlation (method 1)
    max_lag = 10
    bp = boxpierce(adj_x, lags=max_lag)
    Q = bp / (N*max_lag)
    fQ = f2_transformation(Q,7.53,0.103)
    # Serial correlation (make sure box pierce statistic is returned as well)
    # Method 2, but ends up never finishing :(
    #lbvalue, pvalue, bpvalue, bppvalue = acorr_ljungbox(na_contiguous(adj_x), lags=max_lag, boxpierce=True)
    # The above returns values for each lag, so just grab the final value
    #Q = bpvalue[-1] / (N*max_lag)
    #fQ = f2_transformation(Q,7.53,0.103)

    # Nonlinearity (add try/except block to capture USER IDs where this doesn't work)
    # (THIS REQUIRES THE TIMESERIES OBJECT VERSION OF OUR DATA)
    adj_x_contiguous = r.ts(FloatVector(na_contiguous(adj_x)))
    non_linear_test = rtseries.terasvirta_test_ts(adj_x_contiguous,type = "Chisq")
    
    # Grab first element like we did for untransformed data
    p = list(non_linear_test[0])[0]
    fp = f1_transformation(p,0.069,2.304)
    
    # Skewness
    skew = abs(np.mean((t_adj_x.dropna() - xbar) ** 3)/(std ** 3))
    fs = f1_transformation(skew,1.510,5.993)

    # Kurtosis
    kurtosis = np.mean((t_adj_x.dropna() - xbar) ** 4)/(std ** 4)
    fk = f1_transformation(kurtosis,2.273,11567)
    
    measures_list = measures + [fQ,fp,fs,fk]

    return measures_list

## Process and Combine WESAD Data

The WESAD data came in zipped files by subject, with processed data pickled together and unprocessed data contained within the same zipped file per subject.

Here I will grab the processed data and create a dictionary of subject data by subject id:

In [7]:
# This is from the data_etl.py script
# Posting the format of the data below from the wesad_readme.pdf
"""
According to the README:
The double-tap signal pattern was used to manually synchronise the two devices' raw data. The result is provided in the files SX.pkl, one file per subject. This file is a dictionary, with the following keys:
- 'subject': SX, the subject ID
- 'signal': includes all the raw data, in two fields:
  - 'chest': RespiBAN data (all the modalities: ACC, ECG, EDA, EMG, RESP, TEMP)
  - 'wrist':EmpaticaE4data(all the modalities:ACC,BVP,EDA,TEMP)
- 'label': ID of the respective study protocol condition, sampled at 700 Hz. The following IDs
are provided: 0 = not defined / transient, 1 = baseline, 2 = stress, 3 = amusement, 4 = meditation, 5/6/7 = should be ignored in this dataset
"""

# Study protocal conditions (label) mapping
label_map = {
    0: 'not defined / transient',
    1: 'baseline',
    2: 'stress',
    3: 'amusement',
    4: 'meditation',
}

import os
import glob
import pickle
# Read in WESAD datasets by subject and unpickle
subject_dct = {}
path = '../../data/WESAD'
filenames = glob.glob(os.path.join(path,'*/*.pkl'))
for file in filenames:
    
    # Had to use 'latin1' as the encoding due to Python 2/3 pickle incompatibility
    # https://stackoverflow.com/questions/11305790/pickle-incompatibility-of-numpy-arrays-between-python-2-and-3
    unpickled_file = pickle.load(open(file,'rb'), encoding='latin1')
    # Grab relevant info
    subject_id = unpickled_file['subject']
    print('processing subject',subject_id)
    # Grab chest and wrist dataframes
    chest_dct = unpickled_file['signal']['chest']
    wrist_dct = unpickled_file['signal']['wrist']

    # Process the chest dictionary first as it is more straight forward
    # Since the 'ACC' column contains 3 dimensional tuples, it needs to be processed separately due to pandas expecting the same format for all columns
    # Going to create dictionaries without that column to turn into a dataframe, then add the 'ACC' values later
    tmp_chest_dct = dict((k, chest_dct[k].ravel()) for k in list(chest_dct.keys()) if k not in ['ACC'])
    tmp_chest_df = pd.DataFrame(tmp_chest_dct) # Contains everything except ACC
    tmp_acc_df = pd.DataFrame(chest_dct['ACC'],columns=['ACC_X','ACC_Y','ACC_Z']) # Manually declare keys, otherwise shows up as 0,1,2
    final_chest_df = pd.concat([tmp_chest_df,tmp_acc_df],axis=1)

    # Process wrist dictionary, which will take more care because the samplying frequencies were different 
    # Meaning the number of data points collected for each feature is different (higher frequency equals more data points)
    # Basically this one just needs to be processed manually
    wrist_acc_df = pd.DataFrame(wrist_dct['ACC'],columns=['ACC_X','ACC_Y','ACC_Z'])
    wrist_bvp_df = pd.DataFrame(wrist_dct['BVP'],columns=['BVP'])
    wrist_eda_df = pd.DataFrame(wrist_dct['EDA'],columns=['EDA'])
    wrist_temp_df = pd.DataFrame(wrist_dct['TEMP'],columns=['TEMP'])

    # Add labels as a separate object to be returned
    # While the time granularity is the same as the chest data, I'm not sure yet how to use it with the wrist data
    # So will just keep it separate and add as needed
    labels_df = pd.DataFrame(unpickled_file['label'],columns=['label'])
    labels_df['mapped_label'] = labels_df['label'].map(label_map)
    labels_df['SUBJECT_ID'] = subject_id
    
    # Add subject id to all dataframes
    for df in [final_chest_df, wrist_acc_df, wrist_bvp_df, wrist_eda_df, wrist_temp_df]:
        df['SUBJECT_ID'] = subject_id

    subject_dct[subject_id] = {
        'chest_df': final_chest_df,
        'wrist_dfs': {
            'wrist_acc_df': wrist_acc_df,
            'wrist_bvp_df': wrist_bvp_df,
            'wrist_eda_df': wrist_eda_df,
            'wrist_temp_df': wrist_temp_df,
        },
        'labels': labels_df,
    }

processing subject S5
processing subject S2
processing subject S3
processing subject S4
processing subject S17
processing subject S10
processing subject S11
processing subject S16
processing subject S8
processing subject S6
processing subject S7
processing subject S9
processing subject S13
processing subject S14
processing subject S15


## Chest Data + Feature Extraction

According to the README all data was sampled at 700 Hertz. In order to get the period from this, we take the reciprocal of the frequency, which is 1/700. However, looking at the docs for the [sm.tsa.seasonal_decompose](https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.seasonal_decompose.html) and [pandas DatetimeIndex](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) functions, it won't accept a non integer period value. So I guess we have to input the period as 1...

In [8]:
subject_dct['S6']['chest_df'].describe()

,ECG,EMG,EDA,Temp,Resp,ACC_X,ACC_Y,ACC_Z
count,4.949700e+06,4.949700e+06,4.949700e+06,4.949700e+06,4.949700e+06,4.949700e+06,4.949700e+06,4.949700e+06
mean,6.865652e-04,-3.929084e-03,8.292531e+00,3.396810e+01,5.261134e-02,7.101258e-01,-5.102462e-02,-4.128740e-01
std,3.035570e-01,1.791825e-02,1.534487e+00,3.257616e-01,4.119917e+00,1.245003e-01,5.831174e-02,4.362070e-01
min,-7.138824e-01,-4.449463e-01,5.171204e+00,3.287875e+01,-3.152313e+01,2.490000e-01,-6.728000e-01,-3.468600e+00
25%,-9.095764e-02,-1.158142e-02,7.506561e+00,3.372012e+01,-2.288818e+00,5.962000e-01,-8.520001e-02,-7.426000e-01
50%,-2.494812e-02,-3.936768e-03,8.231735e+00,3.398135e+01,-1.205444e-01,7.006000e-01,-4.439998e-02,-6.246000e-01
75%,2.009583e-02,3.524780e-03,9.099960e+00,3.421146e+01,2.381897e+00,8.290000e-01,-1.639998e-02,-2.286000e-01
max,1.499954e+00,4.730988e-01,1.225357e+01,3.522610e+01,3.880005e+01,1.844200e+00,5.590000e-01,2.284200e+00


In [9]:
# Pick a column we want to analyze
# Some of the columns won't make sense to cluster, such as accelerometer
analysis_col = 'EDA'
chest_measures_dct = {}
for id,dfs in subject_dct.items():
    print('Now calculating chest measures for user ',id)
    chest_df = dfs['chest_df'][analysis_col]
    try:
        chest_measures_dct[id] = calculate_measures(chest_df)
    except Exception as e:
        print('Code hit an error for the following reason:\n',e)

Now calculating chest measures for user  S5
Frequency calculated
Decomposition complete
Adj x complete
Calculating trend/seasonal measures
Calculating serial correlation
Calculating non linear test
Calculating skew + kurtosis
Calculting hurst
Calculating Lyapunov Exponent
Calculating adjusted data measures
Now calculating chest measures for user  S2
Frequency calculated
Decomposition complete
Adj x complete
Calculating trend/seasonal measures
Calculating serial correlation
Calculating non linear test
Calculating skew + kurtosis
Calculting hurst
Calculating Lyapunov Exponent
Calculating adjusted data measures
Now calculating chest measures for user  S3
Frequency calculated
Code hit an error for the following reason:
 Data must be positive.
Now calculating chest measures for user  S4
Frequency calculated
Decomposition complete
Adj x complete
Calculating trend/seasonal measures
Calculating serial correlation
Calculating non linear test
Calculating skew + kurtosis
Calculting hurst
Calculat

In [19]:
# Let's see for which subjects the above code did not work for whatever reason
[key for key in subject_dct.keys() if key not in chest_measures_dct.keys()]

['S3', 'S10', 'S14']

In [20]:
# Not too bad, let's inspect our features from the data we DID get
chest_measures_df = pd.DataFrame.from_dict(chest_measures_dct,orient='index',columns=["frequency", "trend","seasonal", "autocorrelation","non-linear","skewness","kurtosis","Hurst","Lyapunov","dc autocorrelation","dc non-linear","dc skewness","dc kurtosis"])
display(chest_measures_df)

,frequency,trend,seasonal,autocorrelation,non-linear,skewness,kurtosis,Hurst,Lyapunov,dc autocorrelation,dc non-linear,dc skewness,dc kurtosis
S5,0.0,1,0,1.0,84.016253,0.002917,0.006312,0.999954,None,1.0,-0.002234,0.335249,0.000752
S2,0.0,1,0,1.0,4010.111120,0.827380,0.999915,0.999954,None,1.0,0.001490,0.335249,0.000752
S4,0.0,1,0,1.0,5847.291864,0.390720,0.139011,0.999954,None,1.0,0.002052,0.335249,0.000752
S17,0.0,1,0,1.0,549.117971,0.001004,0.002709,0.999954,None,1.0,0.000308,0.335249,0.000752
S11,0.0,0,0,1.0,1382.164930,0.475247,0.716278,0.999954,None,1.0,0.000021,0.335249,0.000752
S16,0.0,1,0,1.0,735.898481,0.406702,0.289871,0.999954,None,1.0,0.001037,0.335249,0.000752
S8,0.0,0,0,1.0,2321.617123,0.698201,0.999985,0.999954,None,1.0,-0.002416,0.335249,0.000752
S6,0.0,1,0,1.0,18.728807,0.080102,0.044070,0.999954,None,1.0,-0.001148,0.335249,0.000752
S7,0.0,1,0,1.0,59.791605,0.437424,0.089567,0.999954,None,1.0,0.000187,0.335249,0.000752
S9,0.0,1,0,1.0,778.194924,0.057255,0.010862,0.999954,None,1.0,-0.000024,0.335249,0.000752


## Wrist Data + Feature Extraction

In [12]:
# Pick a column we want to analyze
# Some of the columns won't make sense to cluster, such as accelerometer
analysis_col = 'EDA'
errors = {}
wrist_measures_dct = {}
for id,dfs in subject_dct.items(): 
    print('Now calculating wrist measures for user ',id)
    wrist_total_df = pd.concat([dfs['wrist_dfs'][key] for key in dfs['wrist_dfs']],axis=1)
    try:
        wrist_measures_dct[id] = calculate_measures(wrist_total_df[analysis_col])
    except Exception as e:
        print('\nFollowing exception occurred:\n',e)
        errors[id] = e

Now calculating wrist measures for user  S5
Frequency calculated
Decomposition complete
Adj x complete
Calculating trend/seasonal measures
Calculating serial correlation
Calculating non linear test
Calculating skew + kurtosis
Calculting hurst
Calculating Lyapunov Exponent
Calculating adjusted data measures


<ipython-input-6-ccc57babc47d>:188: RuntimeWarning: invalid value encountered in double_scalars
  skew = abs(np.mean((t_adj_x.dropna() - xbar) ** 3)/(std ** 3))
<ipython-input-6-ccc57babc47d>:192: RuntimeWarning: invalid value encountered in double_scalars
  kurtosis = np.mean((t_adj_x.dropna() - xbar) ** 4)/(std ** 4)


Now calculating wrist measures for user  S2
Frequency calculated
Decomposition complete
Adj x complete
Calculating trend/seasonal measures
Calculating serial correlation
Calculating non linear test


/Users/nfritter/workspace/fitbit/data-science/gitRepos/Characteristic-Based-Time-Series-Clustering/venv/lib/python3.8/site-packages/pypr/stattest/ljungbox.py:38: RuntimeWarning: invalid value encountered in double_scalars
  return N/D
R[write to console]: Error in lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) : 
  0 (non-NA) cases

R[write to console]: In addition: 
R[write to console]: There were 11 warnings (use warnings() to see them)
R[write to console]: 



Calculating skew + kurtosis
Calculting hurst
Calculating Lyapunov Exponent
Calculating adjusted data measures

Following exception occurred:
 Error in lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) : 
  0 (non-NA) cases

Now calculating wrist measures for user  S3
Frequency calculated
Decomposition complete
Adj x complete
Calculating trend/seasonal measures
Calculating serial correlation
Calculating non linear test
Calculating skew + kurtosis
Calculting hurst
Calculating Lyapunov Exponent
Calculating adjusted data measures


...

Now calculating wrist measures for user  S9
Frequency calculated
Decomposition complete
Adj x complete
Calculating trend/seasonal measures
Calculating serial correlation
Calculating non linear test
Calculating skew + kurtosis
Calculting hurst
Calculating Lyapunov Exponent
Calculating adjusted data measures
Now calculating wrist measures for user  S13
Frequency calculated
Decomposition complete
Adj x complete
Calculating trend/seasonal measures
Calculating serial correlation
Calculating non linear test
Calculating skew + kurtosis
Calculting hurst
Calculating Lyapunov Exponent
Calculating adjusted data measures


/Users/nfritter/workspace/fitbit/data-science/gitRepos/Characteristic-Based-Time-Series-Clustering/venv/lib/python3.8/site-packages/pypr/stattest/ljungbox.py:38: RuntimeWarning: invalid value encountered in double_scalars
  return N/D


Now calculating wrist measures for user  S14
Frequency calculated
Decomposition complete
Adj x complete
Calculating trend/seasonal measures
Calculating serial correlation
Calculating non linear test
Calculating skew + kurtosis
Calculting hurst
Calculating Lyapunov Exponent
Calculating adjusted data measures


R[write to console]: Error in lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) : 
  0 (non-NA) cases

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: unable to compute correlation matrix; maybe change 'h' 




Following exception occurred:
 Error in lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) : 
  0 (non-NA) cases

Now calculating wrist measures for user  S15
Frequency calculated
Decomposition complete
Adj x complete
Calculating trend/seasonal measures
Calculating serial correlation
Calculating non linear test
Calculating skew + kurtosis
Calculting hurst
Calculating Lyapunov Exponent
Calculating adjusted data measures


In [13]:
# Check how many subjects incurred errors
display(errors)

{'S2': rpy2.rinterface_lib.embedded.RRuntimeError('Error in lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) : \n  0 (non-NA) cases\n'),
 'S3': rpy2.rinterface_lib.embedded.RRuntimeError('Error in lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) : \n  0 (non-NA) cases\n'),
 'S16': rpy2.rinterface_lib.embedded.RRuntimeError('Error in lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) : \n  0 (non-NA) cases\n'),
 'S8': rpy2.rinterface_lib.embedded.RRuntimeError('Error in lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) : \n  0 (non-NA) cases\n'),
 'S14': rpy2.rinterface_lib.embedded.RRuntimeError('Error in lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) : \n  0 (non-NA) cases\n')}

In [14]:
# Looks like all the same errors, that might be addressable
# Let's see our features
wrist_measures_df = pd.DataFrame.from_dict(wrist_measures_dct,orient='index',columns=["frequency", "trend","seasonal", "autocorrelation","non-linear","skewness","kurtosis","Hurst","Lyapunov","dc autocorrelation","dc non-linear","dc skewness","dc kurtosis"])
display(wrist_measures_df)

,frequency,trend,seasonal,autocorrelation,non-linear,skewness,kurtosis,Hurst,Lyapunov,dc autocorrelation,dc non-linear,dc skewness,dc kurtosis
S5,0.0,1,0,0.026349,2.311066,0.507419,0.486695,0.999954,None,0.352655,5.572116e-08,NaN,NaN
S4,0.0,1,0,0.026364,44.481694,0.804111,0.996398,0.999954,None,NaN,-1.132397e-09,0.335249,0.000752
S17,0.0,1,0,0.026201,21.400773,0.856079,0.999997,0.999954,None,0.352640,-5.460976e-08,NaN,NaN
S10,0.0,1,0,0.026357,3.165258,0.473951,0.281150,0.999954,None,0.352620,-5.308065e-08,NaN,NaN
S11,0.0,1,0,0.026317,13.016022,0.434946,0.231958,0.999954,None,NaN,-7.905507e-08,0.335249,0.000752
S6,0.0,1,0,0.026365,6.650583,0.260931,0.028975,0.999954,None,0.352684,-8.482500e-08,0.335249,0.000752
S7,0.0,1,0,0.026357,37.922325,0.272355,0.023140,0.999954,None,NaN,-7.619502e-08,0.335249,0.000752
S9,0.0,1,0,0.026357,140.871185,0.283348,0.027142,0.999954,None,0.352605,3.664775e-10,0.335249,0.000752
S13,0.0,1,0,0.026363,17.277814,0.358343,0.088192,0.999954,None,NaN,-8.852119e-10,0.335249,0.000752
S15,0.0,1,0,0.026344,1.635552,0.334374,0.055538,0.999954,None,0.352606,-4.240224e-09,0.335249,0.000752


## Optional: Scaling (Standardizing) the Raw Features via Alternative Methods

Given the guidelines of the paper, certain features should be within certain ranges of values. When this is not feasible, alternative effective scaling methods were also mentioned.

The alternative scaling methods (albeit whose purpose was to be be used for comparison in clustering experiments later in the paper):
  - A linear transform method, mapping (–∞,∞) to [0,1]   range:
\begin{equation*} vnorm = \frac{v_i − min(v_1...v_n)}{max(v_1...v_n)−min(v_1...v_n)} \end{equation*}
    - Where vnorm is the normalized value and v_i is a instance value of actual values
  - The Softmax scaling (the logistic function) to map (–∞,∞) to (0, 1):
\begin{equation*} v_n = \frac{1}{1+e^{−v_i}} \end{equation*}
where \begin{equation*} e^{−v_i} = 1/e^{v_i} \end{equation*} v_n denotes the normalized value and v_i denotes a instance value of actual values 

Let's start with the first method, the Linear Transformation

In [15]:
def linear_transformation(df):
    # Take in an input dataframe, take each value in every column and apply the linear transformation column wise
    # Because pandas operates column wise, we can use the following one liner
    linear_transformed_data = ((df - df.min())/(df.max() - df.min()))
    if linear_transformed_data['ANALYSIS_PERIODICTY_RAW'].isnull().sum() == len(linear_transformed_data):
        # This means all values got coerced to nulls; will set back to 1 here
        print('Converting null values back to 1')
        linear_transformed_data['ANALYSIS_PERIODICTY_RAW'] = 1

    return linear_transformed_data

import math
def softmax_scaling(df):
    # Take in an input dataframe, take each value in every column and apply the softmax transformation column wise
    # Because pandas operates column wise, we can use the following one liner
    #softmax_scaled_data = df.apply(lambda x: 1 / (1 + math.exp(-pd.to_numeric(x))))
    softmax_scaled_data = pd.DataFrame()
    for col in df.columns:
        softmax_scaled_data_col = df[col].apply(lambda x: (1 / (1 + math.exp(-x))))
        softmax_scaled_data = pd.concat([softmax_scaled_data,softmax_scaled_data_col],axis=1)
    return softmax_scaled_data

# Clustering!!!
Finally, to the good stuff!

Now that we have our scaled data, let's go ahead and apply some clustering methods to see if the subjects can be clustered together via extracted time series features.

According to the paper, two methods were used: heirarchical clustering and self organizing map clustering. Hierachical clutsering is a well known method that has been applied in many domains, and SOMs are robust in parameter selection, natural clustering results, and achieve superior visualization compared to other clustering methods, such as hierarchical clustering and K-means.

Let's move forward with both methods

# Hierachical Clustering
From the paper:

"There are three major variants of hierarchical clustering algorithms: Single-link, complete-link, and minimum-variance algorithms. Of these three, the single-link and complete-link algorithms are most popular."

"We have chosen the complete-link hierarchical clustering algorithm to achieve more useful hierarchies than single-link from a pragmatic viewpoint."

According to python scikit-learn docs (https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering) the four major kinds of hierarchical clustering:
  - Ward minimizes the sum of squared differences within all clusters. 
    - It is a variance-minimizing approach and in this sense is similar to the k-means objective function but tackled with an agglomerative hierarchical approach.
  - Maximum or complete linkage minimizes the maximum distance between observations of pairs of clusters.
  - Average linkage minimizes the average of the distances between all observations of pairs of clusters.
  - Single linkage minimizes the distance between the closest observations of pairs of clusters.
Looks like the "average linkage" method gets no love...

In [16]:
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import dendrogram

def hierarchical_clustering_sklearn(normalized_df,n_clusters=2,affinity='euclidean',linkage='complete'):
    clustering = AgglomerativeClustering(n_clusters=n_clusters,affinity=affinity,linkage=linkage).fit(normalized_df)
    print('Number of clusters: ',clustering.n_clusters_)
    print('Number of leaves: ',clustering.n_leaves_)
    print('Number of connected components: ',clustering.n_connected_components_)
    return clustering

def hierarchical_clustering_scipy(normalized_df,method='complete',metric='euclidean'):
    clustering = sch.linkage(normalized_df,method=method,metric=metric)
    return clustering

def plot_dendrogram_scipy(clustering_model):
    dendrogram = sch.dendrogram(clustering_model)
    plt.title('Dendrogram')
    plt.xlabel('Customers')
    plt.ylabel('Euclidean distances')
    plt.show()

def plot_dendrogram_sklearn(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)
    plt.title('Hierarchical Clustering Dendrogram')
    # plot the top three levels of the dendrogram
    plot_dendrogram(model, truncate_mode='level', p=3)
    plt.xlabel("Number of points in node (or index of point if no parenthesis).")
    plt.show()

In [17]:
# Going to use the approach from https://github.com/hhl60492/SOMPY_robust_clustering/blob/master/sompy/examples/main.py
# As well as apply k means clustering to the clusters generated by the above
# Also had to do some workarounds to get sompy package to work: https://github.com/sevamoo/SOMPY/issues/36
# Needed to do the following to get this to work:
# pip3 install git+https://github.com/compmonks/SOMPY.git
# pip3 install ipdb==0.8.1
from sompy.sompy import SOMFactory
from sompy.visualization.mapview import View2D
from sompy.visualization.umatrix import UMatrixView
from sompy.visualization.hitmap import HitMapView
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist


def self_organizing_map(normalized_df,normalization='var',initialization='pca',n_job=1,train_rough_len=2,train_finetune_len=5,verbose=None):
    # create the SOM network and train it. You can experiment with different normalizations and initializations
    som = SOMFactory().build(normalized_df.values,normalization=normalization,initialization=initialization,component_names=normalized_df.columns)
    som.train(n_job=n_job,train_rough_len=train_rough_len,train_finetune_len=train_finetune_len,verbose=verbose)
    
    # The quantization error: average distance between each data vector and its BMU.
    # The topographic error: the proportion of all data vectors for which first and second BMUs are not adjacent units.
    topographic_error = som.calculate_topographic_error()
    quantization_error = np.mean(som._bmu[1])
    print("Topographic error = %s; Quantization error = %s" % (topographic_error, quantization_error))
    return som

def som_component_planes(som):
    # component planes view
    view2D = View2D(15,15,"rand data",text_size=12)
    view2D.show(som, col_sz=4, which_dim="all", desnormalize=True)
    plt.show()

def som_kmeans_clustering_predict(som,k):
    # This performed K-means clustering with k clusters on the SOM grid to PREDICT clusters
    #[labels, km, norm_data] = som.cluster(K,K_opt)
    map_labels = som.cluster(n_clusters=k)
    data_labels = np.array([map_labels[int(k)] for k in som._bmu[0]])
    hits = HitMapView(20,20,"Clustering",text_size=12)
    a=hits.show(som)
    return som,map_labels

def som_kmeans_clustering(som,k):
    # Perform K Means clustering on the SOM grid just FITTING data so we can get more data returned
    kMeansCluster = KMeans(n_clusters=k).fit(
        som._normalizer.denormalize_by(som.data_raw,
                                        som.codebook.matrix))
    return kMeansCluster

def som_u_matrix(som):
    # U-matrix plot
    umat = UMatrixView(width=10,height=10,title='U-matrix')
    umat.show(som)

/Users/nfritter/workspace/fitbit/data-science/gitRepos/Characteristic-Based-Time-Series-Clustering/venv/lib/python3.8/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
CACHEDIR=/Users/nfritter/.matplotlib
Using fontManager instance from /Users/nfritter/.matplotlib/fontlist-v330.json
Loaded backend module://ipykernel.pylab.backend_inline version unknown.
Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [21]:
## Clustering parameters
# Hierarchical Clustering parameters
n_clusters = 3 # Number of clusters for clustering
linkage = 'complete' # Type of hierarchical clustering method
affinity = 'euclidean' # "Proximity" Measure for hierarchical clustering

# Self Organizing Map parameters
normalization = 'var'
initialization = 'pca'
n_job = 2
verbose = 'info'
train_rough_len = 1
train_finetune_len = 5

## Hierarchical Clustering (on regular data)
chest_clustering = hierarchical_clustering_sklearn(chest_measures_df,n_clusters=n_clusters,linkage=linkage)
wrist_clustering = hierarchical_clustering_sklearn(wrist_measures_df,n_clusters=n_clusters,linkage=linkage)

# Self Organizing Map Grid Creation (on regular data)
chest_som = self_organizing_map(chest_measures_df,normalization=normalization,initialization=initialization,n_job=n_job,train_rough_len=train_rough_len,train_finetune_len=train_finetune_len,verbose=verbose)
wrist_som = self_organizing_map(wrist_measures_df,normalization=normalization,initialization=initialization,n_job=n_job,train_rough_len=train_rough_len,train_finetune_len=train_finetune_len,verbose=verbose)

## Hierarchical Clustering (Using Linear + Softmax Transformations)
#clustering_linear = hierarchical_clustering_sklearn(linearly_transformed_df,n_clusters=n_clusters,linkage=linkage)
#clustering_softmax = hierarchical_clustering_sklearn(softmax_scaled_df,n_clusters=n_clusters,linkage=linkage)

# Self Organizing Map Grid Creation (Using Linear + Softmax Transformations)
#som_linear = self_organizing_map(linearly_transformed_df,normalization=normalization,initialization=initialization,n_job=n_job,train_rough_len=train_rough_len,train_finetune_len=train_finetune_len,verbose=verbose)
#som_softmax = self_organizing_map(softmax_scaled_df,normalization=normalization,initialization=initialization,n_job=n_job,train_rough_len=train_rough_len,train_finetune_len=train_finetune_len,verbose=verbose)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

## Visualizing Clusters

There are many methods we can choose to visualize clusters:
  - Dendrograms
  - Elbow Curves
  - SOMs
  - Simply grouping by cluster and showing an aggregate value (mean, median, etc.)

In [ ]:
# https://medium.com/@masarudheena/4-best-ways-to-find-optimal-number-of-clusters-for-clustering-with-python-code-706199fa957c
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
# https://medium.com/@sametgirgin/hierarchical-clustering-model-in-5-steps-with-python-6c45087d4318
# https://www.geeksforgeeks.org/elbow-method-for-optimal-value-of-k-in-kmeans/
from scipy.cluster.hierarchy import dendrogram

def concat_data_and_clusters(X,features,labels):
    subject_ids_and_cluster = pd.concat([pd.Series(features.index),pd.Series(labels)],axis=1)
    subject_ids_and_cluster.columns = ['SUBJECT_ID','CLUSTER']
    x_with_clusters = pd.merge(X,user_ids_and_cluster,how='inner',on='SUBJECT_ID')
    return x_with_clusters

def visualize_cluster(original_time_series_data,time_series_features_processed_df,clustering):
    # https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py
    x_with_clusters = concat_data_and_clusters(original_time_series_data,time_series_features_processed_df,clustering.labels_)
    
    # Create dictionaries for storing results/
    clusters = {}
    summaries = {}
    histograms = {}
    for cluster in x_with_clusters['CLUSTER'].unique():
        current_cluster = x_with_clusters.loc[x_with_clusters['CLUSTER'] == cluster]
        current_cluster_summary = current_cluster.describe()
        current_cluster_histogram = current_cluster.hist(bins=50,figsize=(8,8))
        print(current_cluster_summary)
        clusters[cluster] = current_cluster
        summaries[cluster] = current_cluster_summary
        histograms[cluster] = current_cluster_histogram

def plot_data_with_clusters(original_time_series_data,time_series_features_processed_df,clustering,analysis_column,date_col):
    x_with_clusters = concat_data_and_clusters(original_time_series_data,time_series_features_processed_df,clustering.labels_)
    
    # According to docs, can plot labeled data this way if the labels have the column name 'y'
    '''
    x_with_clusters['y'] = x_with_clusters['CLUSTER']
    x_col = 'DATE_COL'
    y_col = analysis_column[0]
    plt.plot(x_col,y_col,data=x_with_clusters)
    plt.show()
    '''
    x = x_with_clusters[analysis_column]
    dates = x_with_clusters[[date_col]]
    labels = x_with_clusters['CLUSTER']
    all_colors = ['red','green','blue','orange','purple']
    colors = all_colors[:len(set(labels))]
    #plt.scatter(x,y,c=labels)
    plt.scatter(dates.values.ravel().tolist(),x.values.ravel().tolist(),c=labels.values.ravel(),cmap=plt.cm.Spectral)
    plt.show()
    '''
    xi = x_with_clusters[['DATE_COL']]
    yi = x_with_clusters[analysis_col]
    labels = x_with_clusters[['CLUSTER']]
    plt.plot(xi, yi, labels=labels)
    #plt.legend()
    plt.show()
    
    by_label = tmp_df.groupby('CLUSTER')
    for name, group in by_label:
        plt.plot(group['DATE_COL'], group[analysis_col[0]], label=name)

    plt.legend()
    plt.show()
    '''

def plot_2d_clusters(original_time_series_data,time_series_features_processed_df,clustering,columns=[]):
    x_with_clusters = concat_data_and_clusters(original_time_series_data,time_series_features_processed_df,clustering.labels_)
    
    if columns:
        #x_subset = x_with_clusters[['USER_ID','CLUSTER'] + columns]
        # https://stackoverflow.com/questions/42056713/matplotlib-scatterplot-with-legend
        x = x_with_clusters[columns[0]]
        y = x_with_clusters[columns[1]]
        unique = list(set(labels))
        colors = [plt.cm.jet(float(i)/max(unique)) for i in unique]
        for i, u in enumerate(unique):
            xi = [x[j] for j  in range(len(x)-1) if labels[j] == u]
            yi = [y[j] for j  in range(len(x)-1) if labels[j] == u]
            plt.scatter(xi, yi, c=colors[i], label=str(u))
        plt.legend()
        plt.show()
    '''
    
    x=[1,2,3,4]
    y=[5,6,7,8]
    classes = [2,4,4,2]
    unique = list(set(classes))
    colors = [plt.cm.jet(float(i)/max(unique)) for i in unique]
    for i, u in enumerate(unique):
        xi = [x[j] for j  in range(len(x)) if classes[j] == u]
        yi = [y[j] for j  in range(len(x)) if classes[j] == u]
        plt.scatter(xi, yi, c=colors[i], label=str(u))
    plt.legend()

    plt.show()
    '''
def plot_cluster_aggregate_values(original_df,transformed_df,cluster_labels,date_col):
    x_with_clusters = concat_data_and_clusters(original_df,transformed_df,cluster_labels)
    by_label_model = x_with_clusters.groupby('CLUSTER')
    for name, group in by_label_model:
        by_activity_date = group.groupby(date_col)
        cluster_aggs = {}
        for date, inner_group in by_activity_date:
            cluster_mean = inner_group[analysis_col].median()
            cluster_aggs[date] = cluster_mean
        plt.plot(list(cluster_aggs.keys()),list(cluster_aggs.values()),label='Softmax cluster %s' % str(name))

    plt.legend()
    plt.show()
    
def plot_elbow_curve(X,som,method,K_values=range(1,10)):
    # Method via https://www.geeksforgeeks.org/elbow-method-for-optimal-value-of-k-in-kmeans/
    # Takes a SOM grid, trains a K Means clustering algorithm using the different K_values
    distortions = [] 
    inertias = [] 
    mapping1 = {} 
    mapping2 = {} 
    for k in K_values: 
        # Training a KMeans clustering model using the transformed SOM grid
        kMeansClustering = som_kmeans_clustering(som,k=k)
        # Save distortion and inertia values
        distortions.append(sum(np.min(cdist(X, kMeansClustering.cluster_centers_, 
                          'euclidean'),axis=1)) / X.shape[0]) 
        inertias.append(kMeansClustering.inertia_) 

        mapping1[k] = sum(np.min(cdist(X, kMeansClustering.cluster_centers_, 
                     'euclidean'),axis=1)) / X.shape[0] 
        mapping2[k] = kMeansClustering.inertia_
        
        # Save SSE values
        # Compute the L2-norm of the vector difference between each element in cluster n and cluster n’s centroid, and add this to the total SSE
        # The L2 norm that is calculated as the square root of the sum of the squared vector values
        
        ### ADD CODE FOR SSE CALCULATION HERE ###
    
    # Plot elbow curve according to supplied method (either 'distortion' or 'inertia')
    if method == 'inertia':
        plt.plot(K_values, inertias, 'bx-') 
        plt.xlabel('Values of K') 
        plt.ylabel('Inertia') 
        plt.title('The Elbow Method using Inertia') 
    if method == 'distortion':
        plt.plot(K_values, distortions, 'bx-') 
        plt.xlabel('Values of K') 
        plt.ylabel('Distortion') 
        plt.title('The Elbow Method using Distortion') 
    if method == 'sse':
        pass
        
    plt.show()